![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re

In [2]:
# Make a request to a webpage
url = "https://electionspeeches.moadoph.gov.au/speeches"
page = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
table = soup.find('section',attrs={"id" :"speeches"}) 

In [4]:
years = []
incumbents = {'incumbent':[],'elected':[],'name':[],'party':[],'party_color':[],'timeplace':[]}
counter = 0 
def getinfo(sided):
    name  = sided.find('h3') #get the name of person
    # print(name.text.strip()) #format the name 
    incumbents['name'].append(name.text.strip())

    party = sided.find('span') #find party of person     
    # print(party.text.strip()) #format party name 
    incumbents['party'].append(party.text.strip())

    try:
        # print(party.attrs['class'][0]) #find party color 
        incumbents['party_color'].append(party.attrs['class'][0].strip())
    except:
        incumbents['party_color'].append('none')
        # print(None) #fail to find the color 

    date = sided.find('p',attrs={'class' : 'delivered'}) #get place and date 
    # print(date.text.strip()) #formate place and date - will need splitting 
    incumbents['timeplace'].append(date.text.strip())


for row in table.findAll('div',attrs={'class': 'row list-item'}): #access all rows of data 
    for year in row.findAll('div', attrs = {'class':'vs span2'}): #find all years
        y = year.find('span')
        years.append(int(y.text.strip()))
        years.append(int(y.text.strip()))
        # print(y.text.strip())

    # find all incumbent prime ministers
    for left in row.findAll('div', attrs = {'class':'span5 left-speech'}): #access left politician in given year 
        getinfo(left)
        incumbents['elected'].append(0)

    for i in range(0,5):
        for left in row.findAll('div', attrs = {'class':'span5 left-speech elected-'+str(i)}): #access left politician in given year 
            getinfo(left)
            incumbents['elected'].append(1)
    incumbents['incumbent'].append(1)

    # find all NON - incumbent prime ministers
    for right in row.findAll('div', attrs = {'class':'span5 right-speech'}): #access left politician in given year 
        getinfo(right)
        incumbents['elected'].append(0)

    for i in range(0,5):
        for right in row.findAll('div', attrs = {'class':'span5 right-speech elected-'+str(i)}): #access left politician in given year 
                getinfo(right)
                incumbents['elected'].append(1)
    incumbents['incumbent'].append(0)

In [5]:
df = pd.DataFrame(incumbents)

df['year'] = years

df['city'] = df['timeplace'].apply(lambda x: re.findall(r'(?<=Delivered at )[A-Za-z]+',x))
df['city'] = df['city'].apply(lambda x: None if len(x)==0 else x[0])

df['date'] = df['timeplace'].apply(lambda x: re.findall(r'\w+\s\d+[a-z]+[, ]+[\d]{4}',x))
df['date'] = pd.to_datetime(df['date'].apply(lambda x: None if len(x)==0 else x[0]))

df['party'] = df['party'].apply(lambda x: x.replace(r'/'," "))
df['party'] = df['party'].apply(lambda x: x.replace(r'Party'," ").strip())

df['party_color'] = df['party_color'].apply(lambda x: 'red' if 'red' in x else 'blue' )

df.to_csv('AUS_metadata_speeches.csv')

In [6]:
df

,incumbent,elected,name,party,party_color,timeplace,year,city,date
0,1,0,Scott Morrison,Liberal,blue,"Delivered at Brisbane , May 15th, 2022",2022,Brisbane,2022-05-15
1,0,1,Anthony Albanese,Labor,blue,"Delivered at Perth , May 1st, 2022",2022,Perth,2022-05-01
2,1,1,Scott Morrison,Liberal,blue,"Delivered at Melbourne, May 12th, 2019",2019,Melbourne,2019-05-12
3,0,0,Bill Shorten,Labor,red,"Delivered at Brisbane, May 5th, 2019",2019,Brisbane,2019-05-05
4,1,1,Malcolm Turnbull,Liberal,blue,"Delivered at Sydney, June 26th, 2016",2016,Sydney,2016-06-26
...,...,...,...,...,...,...,...,...,...
89,0,0,George Reid,Australian Democratic Union,blue,"Delivered at Sydney, NSW, October 23rd, 1906",1906,Sydney,1906-10-23
90,1,1,Alfred Deakin,Protectionist,blue,"Delivered at Ballarat, Vic, October 29th, 1903",1903,Ballarat,1903-10-29
91,0,0,George Reid,Free Trade,blue,"Delivered at Melbourne, Vic, October 30th, 1903",1903,Melbourne,1903-10-30
92,1,1,Edmund Barton,Protectionist,blue,"Delivered at Maitland, NSW, January 17th, 1901",1901,Maitland,1901-01-17
